In [2]:
#!pip install -q pyomo
#!apt-get install -y -qq coinor-cbc

### 4)

The problem can be modeled as follows

Decision variables :
 $x_1 , x_2 , x_3 , x_4 , x_5$

Objective:\
$\text{maximize} \quad 12x_1 + 14x_2 + 10x_3 + 15x_4 + 14x_5$

S.T\
$\begin{align}
6x_1 + 7.5x_2 + 9.5x_3 + 10x_4 + 12.5x_5 \leq 59\\
4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 73\\
x_1,x_2,x_3,x_4,x_5 \in \mathbb{Z}_{\geq 0} 
\end{align}$

In [3]:
from pyomo.environ import *
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('sample_data/lab4_ex3.csv')

In [7]:
df

,Drone_Model,Carrying_Capacity,Cost,Parking_space
0,MANAS,12,6.0,4.25
1,JAY,14,7.5,3.25
2,VEER,10,9.5,2.50
3,SPASHT,15,10.0,3.45
4,DHAKSH,14,12.5,2.25


In [8]:
df['Cost']

0     6.0
1     7.5
2     9.5
3    10.0
4    12.5
Name: Cost, dtype: float64

In [9]:
n = len(df)
m = 2

cols = np.arange(n)
rows = np.arange(m)

cons_name = ['Cost','Parking_space ']
consrhs = [59,73]

In [10]:
m3 = ConcreteModel()
m3.x = Var(cols, domain = NonNegativeIntegers)
m3.cons = ConstraintList()

for i in rows:
    m3.cons.add(summation(df[cons_name[i]],m3.x)<=consrhs[i])
    
m3.capacity = Objective(expr = summation(df['Carrying_Capacity'],m3.x),sense=maximize)    

In [11]:
solver = SolverFactory('cbc')
opt1 = solver.solve(m3)

### 6)

In [12]:
print('Optimal carrying capacity achievable is ',m3.capacity(),'kgs')

for i in range(n):
    print('The number of {} drones is {:.0f}'.format(df['Drone_Model'][i],m3.x[i]()))

Optimal carrying capacity achievable is  114.0 kgs
The number of MANAS  drones is 6
The number of JAY  drones is 3
The number of VEER  drones is 0
The number of SPASHT drones is 0
The number of DHAKSH drones is 0


### 7)

In [13]:
m3a = m3.clone()
m3a.x.domain = NonNegativeReals

opt2 = solver.solve(m3a)

In [22]:
print('Optimal carrying capacity  achievable for the LP is ',m3a.capacity(),'kgs')
for i in range(n):
    print('The number of {} drones is {}'.format(df['Drone_Model'][i],m3a.x[i]()))

Optimal carrying capacity  achievable for the LP is  117.9999996 kgs
The number of MANAS  drones is 9.8333333
The number of JAY  drones is 0.0
The number of VEER  drones is 0.0
The number of SPASHT drones is 0.0
The number of DHAKSH drones is 0.0


In [15]:
m3a.cons.display()

cons : Size=2
    Key : Lower : Body         : Upper
      1 :  None :   58.9999998 :  59.0
      2 :  None : 41.791666525 :  73.0


### 8)

The solution to MILP cannot be obtained by rounding off solution to LP because , rounding off solution to LP will cause the first constraint to be violated.
Also, rounding off 9.83 and 0  don't give 6 and 3 respectively, which are solutions to the MILP

### 9)

In [16]:
m3c = m3a.clone()

m3c.cons[1].set_value(m3c.cons[1].body<=71)
m3c.cons[2].set_value(m3c.cons[2].body<=87)

opt3 = solver.solve(m3c)

In [17]:
print('Optimal carrying capacity achievable for the new LP is ',m3c.capacity(),'kgs')
for i in range(n):
    print('The number of {} drones is {}'.format(df['Drone_Model'][i],m3c.x[i]()))

Optimal carrying capacity achievable for the new LP is  141.999996 kgs
The number of MANAS  drones is 11.833333
The number of JAY  drones is 0.0
The number of VEER  drones is 0.0
The number of SPASHT drones is 0.0
The number of DHAKSH drones is 0.0


In [18]:


m3b = m3.clone()

m3b.cons[1].set_value(m3b.cons[1].body<=71)
m3b.cons[2].set_value(m3b.cons[2].body<=87)

In [19]:
opt3 = solver.solve(m3b)

In [20]:
print('Optimal carrying capacity achievable for the new MILP is ',m3b.capacity(),'kgs')
for i in range(n):
    print('The number of {} drones is {}'.format(df['Drone_Model'][i],m3b.x[i]()))

Optimal carrying capacity achievable for the new MILP is  138.0 kgs
The number of MANAS  drones is 8.0
The number of JAY  drones is 3.0
The number of VEER  drones is 0.0
The number of SPASHT drones is 0.0
The number of DHAKSH drones is 0.0


### 10)

In [21]:
newrhs = [[67,88],[65,89],[63,90],[61,91]]
milps = [m3.clone() for i in range(4)]
lps = [m3a.clone() for i in range(4)]


for j in range(4):
    milps[j].cons[1].set_value(milps[j].cons[1].body<=newrhs[j][0])
    milps[j].cons[2].set_value(milps[j].cons[2].body<=newrhs[j][1])
    lps[j].cons[1].set_value(lps[j].cons[1].body<=newrhs[j][0])
    lps[j].cons[2].set_value(lps[j].cons[2].body<=newrhs[j][1])
    
    opta = solver.solve(milps[j])
    optb = solver.solve(lps[j])
    
    print('When budget limit is',newrhs[j][0],'and space limit is',newrhs[j][1],':')
    print('\n')
    print('Optimal carrying capacity achievable for MILP is ',milps[j].capacity(),'kgs')
    for i in range(n):
        print('The number of {} drones is {}'.format(df['Drone_Model'][i],milps[j].x[i]()))
    print('\n')
    print('Optimal carrying capacity achievable for LP is ',lps[j].capacity(),'kgs')
    for i in range(n):
        print('The number of {} drones is {}'.format(df['Drone_Model'][i],lps[j].x[i]()))
    print('-'*80)    
    print('\n'*3)
    

When budget limit is 67 and space limit is 88 :


Optimal carrying capacity achievable for MILP is  132.0 kgs
The number of MANAS  drones is 11.0
The number of JAY  drones is 0.0
The number of VEER  drones is 0.0
The number of SPASHT drones is 0.0
The number of DHAKSH drones is 0.0


Optimal carrying capacity achievable for LP is  134.000004 kgs
The number of MANAS  drones is 11.166667
The number of JAY  drones is 0.0
The number of VEER  drones is 0.0
The number of SPASHT drones is 0.0
The number of DHAKSH drones is 0.0
--------------------------------------------------------------------------------




When budget limit is 65 and space limit is 89 :


Optimal carrying capacity achievable for MILP is  126.0 kgs
The number of MANAS  drones is 7.0
The number of JAY  drones is 3.0
The number of VEER  drones is 0.0
The number of SPASHT drones is 0.0
The number of DHAKSH drones is 0.0


Optimal carrying capacity achievable for LP is  129.999996 kgs
The number of MANAS  drones is 10.833333
T

As expected, a decrease in budget limit led to a decrease in number of drones, which in turn led to a decrease in maximum possible carrying capacity. In the original problem, the cost constraint value was closer to its bound than the space constraint value. Hence, we can say that the in the current range of 87 - 91, the changing space is not going to have any effect on the solution.
In fact, on observing the data, we can see that on an average, the magnitude of the cost is more than twice thae magnitude of the space. Therefore, as long as the magnitude of  available space (in m) is more than half the value of magnitude of budget (in crores) , it wont have much of an effect on the solution.

we can see that in all the LPs, the optimal solution is one in which as many MANAS drones as possible are bought. This is because MANAS has the highest capacity to cost ratio. And the LPs have a slightly better optimal solution than corresponding MILP.  
In the MILPs, since the capacity is not always divisible by 6, it is adjusted with JAY drones since it has the second highest capacity to cost ratio. This is why optimal solution is slightly less in the MILP

But we can conclude that whenever the budget limit (in Cr) is a multiple of 6, and the available parking is more than budget, optimal solution of corresponding LP and MILP will be the same. 